# Home Assignment 4 - Sugges_

**Author:** Danis Alukaev <br>
**Email:** d.alukaev@innopolis.university <br>
**Group:** B19-DS-01 <br>

One of the strategies to improve user experience is to provide user with hints, or, otherwise, to autocomplete his queries. Let's consider suggest.

Today we will practice generating suggestions using [Trie](https://en.wikipedia.org/wiki/Trie) data structure (prefix tree), see the example below.

Plan of your homework:

1. Build Trie based on real search query data, provided by AOL company;
2. Generate suggestion based on a trie;
3. Measure suggestion speed;
4. [M] Add spellcheck to suggest.


![image](https://www.ritambhara.in/wp-content/uploads/2017/05/Screen-Shot-2017-05-01-at-4.01.38-PM.png)

## 0. Install Trie data structure support

You are free to use any library implementation of Trie, as well as the one we suggest (read the docs before asking any questions!): https://github.com/google/pygtrie

In [ ]:
!pip install pygtrie

In [1]:
import pygtrie
import requests
import os
import gzip
import pandas as pd
from tqdm.auto import tqdm
import math
from collections import Counter
from pprint import pprint

import warnings
warnings.filterwarnings("ignore")

### 0.1. Check it works and understand the example

In [2]:
t = pygtrie.CharTrie()

# trie can be considered as a form of organizing a set of map
t["this is 1"] = "A"
t["this is 2"] = "B"
t["that is 3"] = "C"

print(t)

# "this" string is present in a set
n = t.has_node('this') == pygtrie.Trie.HAS_VALUE
# "this" prefix is present in a set
s = t.has_node('this') == pygtrie.Trie.HAS_SUBTRIE

print(f"Node = {n}\nSubtree = {s}")

# iterate a subtree
for key, val in t.iteritems("this"):
    print(key, '~', val)

CharTrie(this is 1: A, this is 2: B, that is 3: C)
Node = False
Subtree = True
this is 1 ~ A
this is 2 ~ B


## 1. Build a trie upon a dataset

### 1.1. [5] Read the dataset

Download the [dataset](https://github.com/IUCVLab/information-retrieval/tree/main/datasets/aol) (we provide only the first part of the original data for simplicity (~3.5 mln queries)).

Explore the data, see readme file. Load the dataset. Pass the assert.

In [3]:
def download_dataset(url, filename="aol.txt", data_dir="./data"):
    """Download dataset and save it locally.
    
    Parameters
    ----------
    url : str
        link to dataset
    
    filename : str
        name of file with dataset
    
    data_dir : str
        path to directory where dataset located
    
    Returns
    -------
    success : bool
        whether the dataset was successfully downloaded
    """
    success = False
    try:
        if not os.path.exists(data_dir):
            os.makedirs(data_dir)
        archivename = f"{filename}.gz"
        path = os.path.join(data_dir, archivename)
        with open(path, 'wb') as f:
            r = requests.get(url)
            f.write(r.content)
        with gzip.open(path) as f:
            data = f.read()
        path = os.path.join(data_dir, filename)
        with open(path, 'wb') as f:
            f.write(data)
        success = True
    except:
        print("Something went wrong. Try to use different link.")
    return success
    
url = "https://github.com/IUCVLab/information-retrieval/blob/main/datasets/aol/user-ct-test-collection-01.txt.gz?raw=true"
download_dataset(url)

True

In [4]:
def read_dataset(filename="aol.txt", data_dir="./data"):
    """Read AOL dataset as pandas dataframe.
    
    Parameters
    ----------
    filename : str
        name of file with dataset
    
    data_dir : str
        path to directory where dataset located
    
    Returns
    -------
    df : pandas.DataFrame
        dataframe with AOL dataset
    """
    path = os.path.join(data_dir, filename)
    df = pd.read_csv(path, delimiter = "\t")
    return df

In [5]:
aol_data = read_dataset()

assert aol_data.shape[0] == 3558411, "Dataset size does not match"

In [6]:
aol_data.sample(10)

,AnonID,Query,QueryTime,ItemRank,ClickURL
2381084,8097799,mapquest,2006-05-20 09:58:51,NaN,NaN
3487027,22839090,google,2006-05-28 20:20:21,NaN,NaN
2160079,6781296,printable airsoft targets,2006-05-21 19:22:07,1.0,http://www.airsoft-guns.co.uk
3435648,21245737,zachery construction company,2006-05-08 23:13:31,NaN,NaN
361859,883390,marriagemindedpeoplemeet,2006-05-02 17:26:17,NaN,NaN
1256144,3149842,coupon codes,2006-03-02 10:47:09,5.0,http://www.coupon-codes.net
1216084,3042841,yahoo,2006-04-28 15:44:15,2.0,http://search.yahoo.com
37262,91602,spinal facet fusion via injection,2006-04-02 17:22:17,13.0,http://www.aafp.org
2377443,8076981,haverhill country club,2006-05-03 07:14:56,1.0,http://www.haverhillcc.com
883318,2235233,religious checks,2006-05-05 12:12:12,NaN,NaN


### 1.2. [10] Build a Trie

We want a suggest function to be **non-sensitive to stop words** because we don't want to upset the users if they confuses/omits prepositions. Consider *"public events in Innopolis"* vs *"public events at Innopolis"* or *"public events Innopolis"* - they all mean the same.

Build a Trie based on the dataset, **storing query statistics such as query _frequency_, urls and ranks in the nodes**. Some queries may have no associated urls, others may have multiple ranked urls. Think of the way to store this information.

Pass the asserts.

In [7]:
class ExtendedList(list):
    """List with extended properties."""
    
    def __init__(self, *args):
        super().__init__(args)
        self._init_properties(args)
    
    def _init_properties(self, args):
        """Initialize additional properties. 
        
        Supposed that for each new values of list, it
        will be reinitialized, i.e. work as a tuple.
        Expected for elements: frequency, list of urls,
        list of ranks, and list or original queries.
        
        Parameters
        ----------
        args : objects
            objects to store in the list
        
        Returns
        -------
        None
        """
        self.frequency = args[0]
        self.urls = args[1]
        self.ranks = args[2]
        self.queries = args[3]

In [8]:
def build_aol_trie(aol_data, stops):
    """Build a trie based on AOL dataset.
    
    Iteratively adds queries to the trie. In case if query is already 
    in the trie, adds new urls and ranks to the node. Excludes stop words 
    from the query. Utilizes ExtendedList for nodes. The attributes are 
    frequency, list of urls, list of ranks and list of original queries (e.g. 
    useful in case if propositions in similar queries vary). 
    
    Parameters
    ----------
    aol_data : pandas.DataFrame
        AOL dataset
    
    stops : set
        set of stop words
    
    Returns
    -------
    trie : pygtrie.CharTrie
        trie based on AOL dataset
    """
    trie = pygtrie.CharTrie()
    data = aol_data.reset_index()
    freqs = data.Query.value_counts()
    
    convert_nan = lambda x: None if x != x else x
    
    for idx, row in tqdm(data.iterrows(), total=data.shape[0]):
        query = row.Query
        if query != query:
            continue
        freq = convert_nan(freqs[query])
        queries = [query[:]]
        query = " ".join([w for w in query.split() if w not in stops])
        
        urls = [convert_nan(row.ClickURL)]
        ranks = [convert_nan(row.ItemRank)]
        
        if query in trie:
            _, _urls, _ranks, _queries = trie[query]
            urls, ranks, queries = _urls + urls, _ranks + ranks, _queries + queries
        trie[query] = ExtendedList(freq, urls, ranks, queries)
    return trie

stops = set('a on at of to is from for and with using the in &'.split())
aol_trie = build_aol_trie(aol_data, stops)

  0%|          | 0/3558411 [00:00<?, ?it/s]

In [9]:
# test trie
bag = []
for key, val in aol_trie.iteritems("sample q"):
    print(key, '~', val)
    
    #NB: here we assume you store urls in a property of list type. But you can do something different. 
    bag += val.urls
    
    assert "sample question" in key, "All examples have `sample question` substring"
    assert key[:len("sample question")] == "sample question", "All examples have `sample question` starting string"

for url in ["http://www.surveyconnect.com", "http://www.custominsight.com", 
            "http://jobsearchtech.about.com", "http://www.troy.k12.ny.us",
            "http://www.flinders.edu.au", "http://uscis.gov"]:
    assert url in bag, "This url should be in a try"

sample question surveys ~ [5, ['http://www.surveyconnect.com', 'http://www.custominsight.com', 'http://www.askemployees.com', 'http://www.lg-employers.gov.uk', None], [7.0, 4.0, 10.0, 1.0, None], ['sample question surveys', 'sample question surveys', 'sample question surveys', 'sample question surveys', 'sample question surveys']]
sample questions immigration interview ~ [1, [None], [None], ['sample questions for immigration interview']]
sample questions interview ~ [1, ['http://www.quintcareers.com'], [1.0], ['sample questions for interview']]
sample questions family interview ~ [3, ['http://www.grandparents-day.com', 'http://www.quintcareers.com', 'http://jobsearchtech.about.com'], [2.0, 5.0, 3.0], ['sample questions for family interview', 'sample questions for family interview', 'sample questions for family interview']]
sample questions sociology race ethnicity ~ [1, [None], [None], ['sample questions sociology race and ethnicity']]
sample questions biology ~ [2, ['http://www.utexas

## 2. [15] Write a suggest function which is non-sensitive to stop words

Suggest options for user query based on Trie you just built.
Output results sorted by frequency, print query count for each suggestion. If there is an url available, print the url too. If multiple url-s are available, print the one with the highest rank (the less the better).

Pass the asserts.

Question for analysis: What is the empirical threshold for minimal prefix for suggest?

In [10]:
stops = set('a on at of to is from for and with using the in &'.split())

#### ! Important

While using `Python 3.8.12` there is a problem with typing `list[str]`. For this reason I'm importing type `List` from typing and change the return type to `List[str]`.

In [11]:
from typing import List

def complete_user_query(query: str, trie, top_k=5, verbose=False) -> List[str]:
    """Complete user query using Trie.
    
    Suggests top k options for a user query. Retrieves possible
    options from the Trie. Sorts them by frequency and takes k most
    frequent. Outputs to stdout the query count, suggestion, url with 
    highest rank, and rank itself. 
    
    Parameters
    ----------
    query : str
        user query
    
    trie : pygtrie.CharTrie
        Trie used for completion
    
    top_k : int (default: 5)
        number of suggestions to retrieve
    
    verbose : bool (default: False)
        whether output the meta data such as query count, 
        suggestion, url with highest rank, and rank itself
    
    Returns
    -------
    completions : list
        list of user query completion options sorted
        by frequency
    
    """
    _query = query[:]
    for stop in stops:
        _query.replace(stop, "")
    _query.replace("  ", " ")
    
    items = list(trie.iteritems(_query))
    suggestions = sorted(items, key=lambda x: x[1][0], reverse=True)
    top_k = min(top_k, len(suggestions))
    top = suggestions[:top_k]
    
    if not verbose:
        meta = {}
        for idx, suggestion in enumerate(top):
            freq, urls, ranks, queries = suggestion[1]
            query = Counter(queries).most_common(1)[0][0]
            
            meta[idx + 1] = {
                "Suggestion": query,
                "Query Count": freq, 
            }
            ranked_urls = list(zip(urls, ranks))
            ranked_urls = [p for p in ranked_urls if p[0] is not None and p[1] is not None]
            ranked_urls = sorted(ranked_urls, key=lambda x: x[1])
            if len(ranked_urls) > 0:
                meta[idx + 1]["URL"] = ranked_urls[0][0]
                meta[idx + 1]["URL rank"] = ranked_urls[0][1]
        print("\n---------OUTPUT---------")
        pprint(meta)
        print("-------END OUTPUT-------\n")

    completions = [Counter(s[1][3]).most_common(1)[0][0] for s in top]
    return completions

In [12]:
inp = "trie"
print("Query:", inp)
print("Results:")
res = complete_user_query(inp, aol_trie)
print(res)

#NB we assume you return suggested query string only
assert res[0] == "tried and true tattoo"
assert res[1] == "triest" or res[1] == "triethanalomine"

assert "boys and girls club of conyers georgia" \
            in complete_user_query("boys girls club conyers", aol_trie, 10), "Should be here"

Query: trie
Results:

---------OUTPUT---------
{1: {'Query Count': 5,
     'Suggestion': 'tried and true tattoo',
     'URL': 'http://www.triedntruetattoo.com',
     'URL rank': 1.0},
 2: {'Query Count': 3, 'Suggestion': 'triest'},
 3: {'Query Count': 3,
     'Suggestion': 'triethanalomine',
     'URL': 'http://avalon.unomaha.edu',
     'URL rank': 1.0},
 4: {'Query Count': 2, 'Suggestion': 'tried and failed'},
 5: {'Query Count': 1,
     'Suggestion': "tried and truechildren's consignment sale"}}
-------END OUTPUT-------

['tried and true tattoo', 'triest', 'triethanalomine', 'tried and failed', "tried and truechildren's consignment sale"]

---------OUTPUT---------
{1: {'Query Count': 2,
     'Suggestion': 'boys and girls club of conyers georgia',
     'URL': 'http://www.bgcma.org',
     'URL rank': 1.0}}
-------END OUTPUT-------



## 3. Measure suggest speed ##

### 3.1. [10] Full Trie test

Check how fast your search is working. Consider changing your code if it takes too long on average.

Sucess criterion:
- there is an average and variance for **multiple runs** of the given bucket.
- there is an average and variance for **multiple runs** of naive search in unindexed dataset.

In [ ]:
inp_queries = ["inf", "the best ", "information retrieval", "sherlock hol", "carnegie mell", 
               "babies r", "new york", "googol", "inter", "USA sta", "Barbara "]

#TODO: measure avg execution time (in milliseconds) per query and print it out

## 4. [10] Add spellchecking to your suggest

Try to make your search results as close as possible. Compare top-5 results of each query with top-5 results for corrected.

You can use use [pyspellchecker](https://pypi.org/project/pyspellchecker/) `candidates()` call, or use any other spellchecker implementation.

In [ ]:
def complete_user_query_with_spellchecker(query, trie, top_k=5) -> list[str]:
    #TODO: suggest top_k options for a user query
    # sort results by frequency (!!), 
    # suggest the QUERIES for first k ranked urls if available
    pass

In [ ]:
inp_queries = ["inormation retrieval", "shelrock hol", "carnagie mell", "babis r", "Barrbara "]
inp_queries_corrected = ["information retrieval", "sherlock hol", "carnegie mell", "babies r", "Barbara "]

for q, qc in zip(inp_queries, inp_queries_corrected):
    assert  complete_user_query(qc, trie, 5) == \
            complete_user_query_with_spellchecker(q, trie, 5), "Assert {} and {} give different results".format(q, qc)